<a href="https://colab.research.google.com/github/matteraggi/FineTuningAI/blob/main/Progetto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**INSTALLS**

In [1]:
!pip install --upgrade transformers torch datasets peft tf-keras accelerate bitsandbytes trl evaluate radon zss
# IMPORTANTE! RIAVVIARE IL RUNTIME DOPO L'ESECUZIONE

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 40.6 MB/s eta 0:00:00
  

**IMPORTS**

In [1]:
# Importing stock ml libraries
import numpy as np
import pandas as pd
import torch
import gc
from tabulate import tabulate
from datasets import load_dataset
from trl import SFTTrainer
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model

**Svuota la cache**

In [2]:
# Garbace collect
gc.collect()
# Svuota la cache
torch.cuda.empty_cache()

**VARIABLES**

In [4]:
# The model that you want to train from the Hugging Face hub
model_name = "bigcode/starcoder2-3b"
# The instruction dataset to use
dataset_name = "bigcode/self-oss-instruct-sc2-exec-filter-50k"
# Fine-tuned model name
new_model = "starcoder2-finetuned"

# Dimension of dataset subset used
dataset_range = 4000

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension. It determines the size and parameter count of the low-rank adaptation
lora_r = 64  # XXXXXXXXXXX
# Alpha parameter for LoRA scaling factor that determines the impact of the low-rank matrices on the original model's output.
# Controls the overall strength of the low-rank adaptation.
lora_alpha = 2*lora_r  # "Often set to 2-4 times lora_r"
# Dropout probability for LoRA layers (considera aumento a 0.1-0.2)
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True
# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"
# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"
# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False  # "Double quantization can sometimes improve performance but increases complexity.  It's often left disabled initially."

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = True
bf16 = False  # CONFERMO CHE NON E' SUPPORTATO SU COLAB

# Number of training epochs
num_train_epochs = 3  # Sembra arrivare a un plateau già alla terza
# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4  # Abbassare in caso di training più lunghi

# Batch size per training e per evaluation
per_device_train_batch_size = per_device_eval_batch_size = 2
# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 4 # Moltiplica la batch_size vera per ottenere quella simulata. Dinimuire per velocità, alzare per stabilità.

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3  # Stabilizza il training, ma rallentandolo. Range ottimale 0.1-0.5

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.1  # Aumentare in caso di overfitting a 0.1-0.2

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1  # '-1' mantains num_train_epochs

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.05 # Aumentare a 0.1 in caso di instabilità eccessiva iniziale XXXXX

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every tot updates steps
save_steps = 100
# Log every tot updates steps
logging_steps = 100
# Evaluation strategy ("no", "epochs", "steps")
eval_strategy = "steps"
# Evaluate the model every tot steps (if the strategy is "steps")
eval_steps = 100

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use; tronca l'input a tot tokens
max_seq_length = 1024 # Solo 0.33% sopra il 1024 di lunghezza nella sezione di dataset usata

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False  # "Can improve efficiency if your dataset has many short sequences."

# Load the entire model on the GPU 0
device_map = {"": 0}

### SCELTA: carica il modello di base per allenarlo o carica il modello finetunato da git per valutarlo

**NON RUNNARE DI DEFAULT**
*- LOAD BASE MODEL*

In [5]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
  load_in_4bit=use_4bit,
  bnb_4bit_quant_type=bnb_4bit_quant_type,
  bnb_4bit_compute_dtype=compute_dtype,
  bnb_4bit_use_double_quant=use_nested_quant,
)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
  model_name,
  quantization_config=bnb_config,
  device_map=device_map
)

# Load base tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/12.1G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.88k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/777k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/442k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.06M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

**NON RUNNARE DI DEFAULT**
*- LOAD FINETUNED MODEL*

In [ ]:
#Da Git:
!apt-get install git
!git clone https://github.com/matteraggi/FineTuningAI.git

#Dal Drive (cambiare path manualmente):
#from google.colab import drive
#drive.mount('/content/drive')
#!cp -r /content/drive/MyDrive/checkpoint_800 /content/

# Percorso alla directory del modello
checkpoint_path = "/content/FineTuningAI/models/SC_Finetuned"

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
  load_in_4bit=use_4bit,
  bnb_4bit_quant_type=bnb_4bit_quant_type,
  bnb_4bit_compute_dtype=compute_dtype,
  bnb_4bit_use_double_quant=use_nested_quant,
)

# Load (partially) finetuned model
model = AutoModelForCausalLM.from_pretrained(
  checkpoint_path,
  device_map=device_map,
  quantization_config=bnb_config,
)

# Load tokenizer from checkpoint
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

# Da qui, continui l'esecuzione tramite le celle seguenti, ovvero quelle di LoRA

**Last model configs and apply LoRA configuration to the model**

In [6]:
# Ultime configurazioni per il modello
model.config.use_cache = False
model.config.pretraining_tp = 1
if gradient_checkpointing:
  model.gradient_checkpointing_enable()  # Attiva gradient checkpointing per ridurre l'uso di memoria

# Load LoRA configuration
# Example configuration for target modules (can vary per model)
peft_config = LoraConfig(
  lora_alpha=lora_alpha,
  lora_dropout=lora_dropout,
  r=lora_r,
  bias="none",
  task_type="CAUSAL_LM",
  target_modules=["q_proj", "v_proj"]  # These are common modules for transformers (queries and values in attention layers)
)

for param in model.parameters():
    if param.dtype in [torch.float32, torch.float64, torch.float16, torch.bfloat16]:  # Check for floating point types
        param.requires_grad = True  # Ensure only floating-point parameters require gradients
#print(any(param.requires_grad for param in model.parameters() if param.dtype in [torch.float32, torch.float64, torch.float16, torch.bfloat16]))  # Should print True
# Apply LoRA using PEFT
model = get_peft_model(model, peft_config)  # Wrap the model with the LoRA configuration
model.eval()

print(model)
# Now, only LoRA layers will have requires_grad=True
#print(any(param.requires_grad for param in model.parameters()))  # This should be True for LoRA layers

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Starcoder2ForCausalLM(
      (model): Starcoder2Model(
        (embed_tokens): Embedding(49152, 3072)
        (layers): ModuleList(
          (0-29): 30 x Starcoder2DecoderLayer(
            (self_attn): Starcoder2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
          

### Caricamento e tokenizzazione dataset

**DATASET LOAD AND PROCESSING**

In [7]:
# Load dataset (you can process it here)
dataset = load_dataset(dataset_name, split="train")
# Use only a random subset of the dataset 'dataset_range'-wide
dataset = dataset.shuffle(seed=42).select(range(dataset_range))
# Split the subset in training and validation
split_dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = split_dataset["train"]
val_dataset = split_dataset["test"]

README.md:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/90.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/50661 [00:00<?, ? examples/s]

In [8]:
#Controlla la lunghezza degli elementi
from statistics import mean, median

lengths = [ len(tokenizer(f"{ex['instruction']} {ex['response']}")['input_ids']) for ex in dataset ]

# Statistiche principali
max_len = max(lengths)
avg_len = mean(lengths)
med_len = median(lengths)
over_1024 = sum(l > 1024 for l in lengths) / len(lengths) * 100
over_2048 = sum(l > 2048 for l in lengths) / len(lengths) * 100

# Stampa risultati
print(f"Lunghezza massima: {max_len} token")
print(f"Lunghezza media: {avg_len:.2f} token")
print(f"Lunghezza mediana: {med_len} token")
print(f"Percentuale > 1024 token: {over_1024:.2f}%")
print(f"Percentuale > 2048 token: {over_2048:.2f}%")

Lunghezza massima: 1634 token
Lunghezza media: 329.47 token
Lunghezza mediana: 303.0 token
Percentuale > 1024 token: 0.33%
Percentuale > 2048 token: 0.00%


In [9]:
# Inspect dataset columns
# print(dataset.column_names)

# Funzione di preprocessamento migliorata
def preprocess_function(examples):
  input_texts = [f"{instr} {response}" if instr else response for instr, response in zip(examples["instruction"], examples["response"])]
  encodings = tokenizer(input_texts, truncation=True, max_length=max_seq_length)
  #encodings = tokenizer(input_texts)  #, truncation=False, max_length=None)
  encodings["text"] = input_texts  # Aggiungi la colonna "text" per evitare KeyError
  return encodings  # max length aumentata (più efficienza a discapito della memoria) e ora ritorna tensori al posto di lists

# Apply tokenization
##tokenized_dataset = dataset.map(preprocess_function, batched=True)
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_val_dataset = val_dataset.map(preprocess_function, batched=True)

# Inspect dataset columns
# #print(tokenized_dataset.column_names)
# print(tokenized_train_dataset.column_names)
# print(tokenized_val_dataset.column_names)

Map:   0%|          | 0/3200 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

### Training

In [10]:
# Set training parameters
training_arguments = TrainingArguments(
  output_dir=output_dir,
  num_train_epochs=num_train_epochs,
  per_device_train_batch_size=per_device_train_batch_size,
  gradient_accumulation_steps=gradient_accumulation_steps,
  optim=optim,
  eval_strategy=eval_strategy,
  eval_steps=eval_steps,
  save_steps=save_steps,
  logging_steps=logging_steps,
  learning_rate=learning_rate,
  weight_decay=weight_decay,
  fp16=fp16,
  bf16=bf16,
  max_grad_norm=max_grad_norm,
  max_steps=max_steps,
  warmup_ratio=warmup_ratio,
  group_by_length=group_by_length,
  lr_scheduler_type=lr_scheduler_type,
  report_to="tensorboard",
  gradient_checkpointing=gradient_checkpointing,
)

# Train the model with the modified configuration
trainer = SFTTrainer(
  model=model,
  train_dataset=tokenized_train_dataset,
  eval_dataset=tokenized_val_dataset,
  peft_config=peft_config,
  args=training_arguments,
)

Truncating train dataset:   0%|          | 0/3200 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/800 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [11]:
# Start training

# Training da zero
trainer.train()

# Training se già parzialmente allenato
#trainer.train(resume_from_checkpoint=checkpoint_path)

# IGNORE 'Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.' WARNING; CAUSED BY LIBRARY GLITCH

Step,Training Loss,Validation Loss
100,1.454600,0.683269
200,0.828500,0.669694
300,0.782400,0.649764
400,0.763600,0.636099
500,0.727100,0.635959
600,0.711100,0.635858
700,0.710900,0.626262
800,0.705200,0.619176
900,0.683600,0.622405
1000,0.670500,0.618653


TrainOutput(global_step=1200, training_loss=0.7814562543233236, metrics={'train_runtime': 5773.2888, 'train_samples_per_second': 1.663, 'train_steps_per_second': 0.208, 'total_flos': 5.492704318883021e+16, 'train_loss': 0.7814562543233236})

### Save model / checkpoint

**NON RUNNARE DI DEFAULT**
Se ha finito l'allenamento, salva il modello e pushalo sul git

In [12]:
# Save the fine-tuned model
#trainer.model.save_pretrained(new_model)
trainer.save_model("./results/SC_Finetuned")

In [13]:
from google.colab import userdata
!apt-get install git
!git config --global user.email {userdata.get('GitEmail')}
!git config --global user.name {userdata.get('GitUsername')}
!git clone https://github.com/matteraggi/FineTuningAI.git
!cd FineTuningAI  # Go to the *existing* FineTuningAI directory
!mv results/SC_Finetuned FineTuningAI/models/SC_Finetuned/
!cd FineTuningAI && git add models/SC_Finetuned
!cd FineTuningAI && git commit -m "Saved 'SC_Finetuned', new finetuned model. Best one so far, probably the last one"
!git config --global credential.helper store
!cd FineTuningAI && git push https://{userdata.get('PAT')}@github.com/matteraggi/FineTuningAI.git main

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.12).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.
Cloning into 'FineTuningAI'...
remote: Enumerating objects: 260, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 260 (delta 20), reused 19 (delta 4), pack-reused 212 (from 1)
Receiving objects: 100% (260/260), 364.15 MiB | 18.71 MiB/s, done.
Resolving deltas: 100% (135/135), done.
Updating files: 100% (46/46), done.
cp: -r not specified; omitting directory 'results/SC_Finetuned'
On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
Everything up-to-date


**NON RUNNARE DI DEFAULT**
Salva il checkpoint sul drive (cambiare manualmente il nome della cartella)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp -r /content/results/checkpoint-800 /content/drive/MyDrive/

### HumanEval evaluation

**Caricare il benchmark HumanEval**
Dopo il fine-tuning, devi confrontare il tuo modello con uno pre-addestrato (StarCoder2 senza fine-tuning) e con il modello fine-tunato.

In [ ]:
SYSTEM_PROMPT = "Generate Python code for the following task:\n"

In [5]:
import torch
import json
import gc
import random
import pandas as pd
from tabulate import tabulate
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

# Configurazione
base_model_name = "bigcode/starcoder2-3b"
model_path = "/content/FineTuningAI/models/SC_Finetuned"  # Path del modello fine-tunato
device = "cuda" if torch.cuda.is_available() else "cpu"

# Caricamento del dataset HumanEval e selezione casuale di 10 esempi
humaneval = load_dataset("openai/openai_humaneval", split="test")
sampled_humaneval = random.sample(list(humaneval), 10)

# Configurazione quantizzazione
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# Caricamento del modello base
non_finetuned_model = AutoModelForCausalLM.from_pretrained(
    base_model_name, device_map="auto", quantization_config=quantization_config
).eval()


# Funzione per generare codice con il prompt modificato
def generate_code(model, prompt):
    # Aggiungiamo un contesto iniziale per istruire il modello correttamente
    pre_prompt = "# Complete the function as described below.\n"
    full_prompt = pre_prompt + prompt
    inputs = tokenizer(full_prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            do_sample=True,
            temperature=0.7,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


# Valutazione dei modelli
base_results = []
fine_tuned_results = []

for i, example in enumerate(sampled_humaneval):
    prompt = example['prompt']

    base_code = generate_code(non_finetuned_model, prompt)
    base_results.append({"prompt": prompt, "code": base_code})

    fine_tuned_code = generate_code(model, prompt)
    fine_tuned_results.append({"prompt": prompt, "code": fine_tuned_code})

    print(f"Processed example {i+1}/10")
    gc.collect()

# Salvataggio dei risultati in JSON
results = {"base_results": base_results, "fine_tuned_results": fine_tuned_results}

with open("model_results.json", "w") as f:
    json.dump(results, f, indent=4)

# Creazione e stampa tabella
df = pd.DataFrame([
    {"Prompt": b['prompt'], "Non-Finetuned Code": b['code'], "Finetuned Code": f['code']}
    for b, f in zip(base_results, fine_tuned_results)
])

print(tabulate(df, headers="keys", tablefmt="grid", numalign="left", stralign="left"))


README.md:   0%|          | 0.00/6.52k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/83.9k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/164 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/12.1G [00:00<?, ?B/s]

NameError: name 'tokenizer' is not defined

In [ ]:
# Test con prompt semplice
import torch
import json
import gc
import random
import pandas as pd
from tabulate import tabulate
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

# Configurazione
base_model_name = "bigcode/starcoder2-3b"
model_path = "/content/FineTuningAI/models/SC_Finetuned"  # Path del modello fine-tunato
device = "cuda" if torch.cuda.is_available() else "cpu"

# Caricamento del dataset HumanEval e selezione casuale di 10 esempi
humaneval = load_dataset("openai/openai_humaneval", split="test")
sampled_humaneval = random.sample(list(humaneval), 10)

# Configurazione quantizzazione
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# Caricamento del modello base
non_finetuned_model = AutoModelForCausalLM.from_pretrained(
    base_model_name, device_map="auto", quantization_config=quantization_config
).eval()


def generate_code(model, prompt):
    # Istruzione chiara: solo codice, nessuna spiegazione o testo extra
    instruction = "Complete the Python function. Only output the code, no explanations or additional text.\n\n"
    full_prompt = instruction + prompt

    inputs = tokenizer(full_prompt, return_tensors="pt", truncation=True, max_length=2048).to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            do_sample=False,  # Disabilita il campionamento
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,  # Usa eos_token come pad_token
        )

    # Decodifica l'output
    full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Estrai solo la parte che ci interessa, ossia il codice
    # Eliminare tutto prima del codice effettivo (per esempio, "Solution" e numeri casuali)
    start_index = full_output.find("def ")  # Trova la definizione della funzione
    cleaned_output = full_output[start_index:].strip()

    # Restituisci il codice "pulito"
    return cleaned_output

test_prompt = "def add(a, b):\n    \"\"\"Add two numbers.\"\"\"\n"
print("Base model:", generate_code(non_finetuned_model, test_prompt))
print("Fine-tuned:", generate_code(model, test_prompt))

In [ ]:
import json

# Assuming base_results and fine_tuned_results are your arrays
results = {
    "base_results": base_results,
    "fine_tuned_results": fine_tuned_results
}

# Save as JSON
with open("model_results.json", "w") as f:
    json.dump(results, f, indent=4)



In [ ]:
import pandas as pd
from tabulate import tabulate

table_data = []
for non_fine_tuned, fine_tuned in zip(base_results, fine_tuned_results):
    table_data.append({
        "Prompt": non_fine_tuned['prompt'],
        "Non-Finetuned Code": non_fine_tuned['code'],
        "Finetuned Code": fine_tuned['code']
    })
# Use tabulate for better formatting
df = pd.DataFrame(table_data)

# Print only the first 5 rows with tabulate
print(tabulate(df.head(), headers='keys', tablefmt='grid', numalign="left", stralign="left"))

### Save/Load results

**SAVE IN DRIVE**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# Save as JSON in Drive
with open("/content/drive/MyDrive/model_results.json", "w") as f:
    json.dump(results, f, indent=4)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**SAVE IN GIT**

In [ ]:
from google.colab import userdata
import os

# Install git (se non già installato)
!apt-get install git -qq

# Configura Git (modo corretto per Colab)
git_email = userdata.get('GitEmail')
git_username = userdata.get('GitUsername')
pat = userdata.get('PAT')

!git config --global user.email "{git_email}"
!git config --global user.name "{git_username}"

# Clona il repository (se non esiste già o forza il reclone)
if not os.path.exists('FineTuningAI'):
    !git clone https://{pat}@github.com/matteraggi/FineTuningAI.git
else:
    !rm -rf FineTuningAI
    !git clone https://{pat}@github.com/matteraggi/FineTuningAI.git

# Copia i risultati
!cp model_results.json FineTuningAI/results/

# Commit e push
%cd FineTuningAI
!git add results/model_results.json
!git commit -m "Save model results $(date +'%Y-%m-%d %H:%M:%S')"
!git push origin main
%cd ..

Cloning into 'FineTuningAI'...
remote: Enumerating objects: 241, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 241 (delta 9), reused 15 (delta 3), pack-reused 212 (from 1)
Receiving objects: 100% (241/241), 364.12 MiB | 33.41 MiB/s, done.
Resolving deltas: 100% (124/124), done.
Updating files: 100% (46/46), done.
/content/FineTuningAI
[main 424aab1] Save model results 2025-04-06 11:16:46
 1 file changed, 86 insertions(+), 86 deletions(-)
 rewrite results/model_results.json (98%)
Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 2 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (4/4), 4.15 KiB | 4.15 MiB/s, done.
Total 4 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/matteraggi/FineTuningAI.git
   57a9714..424aab1  main -> main
/content


**LOAD FROM DRIVE**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import json

# Load JSON file from Drive
with open("/content/drive/MyDrive/model_results.json", "r") as f:
    results = json.load(f)

# Now results['base_results'] and results['fine_tuned_results'] hold the data
base_results = results['base_results']
fine_tuned_results = results['fine_tuned_results']


Mounted at /content/drive


**LOAD FROM GIT**

In [ ]:
import json
!apt-get install git
!git clone https://github.com/matteraggi/FineTuningAI.git

with open("FineTuningAI/results/model_results.json", "r") as f:
    results = json.load(f)

base_results = results['base_results']
fine_tuned_results = results['fine_tuned_results']

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.12).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.
fatal: destination path 'FineTuningAI' already exists and is not an empty directory.


### **Complex checking** holy shit me vs God he loses

In [ ]:
!pip install pylint

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 522.5/522.5 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.3/275.3 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.2/94.2 kB 9.2 MB/s eta 0:00:00


In [ ]:
import ast
import radon.complexity
import io
import subprocess  # For running the cc command
import json
from pylint import lint
from pylint.reporters.json_reporter import JSONReporter
import tempfile  # For creating a temporary file
import zss
import math
import pandas as pd
import evaluate
from typing import List
import re
import os
os.environ["HF_ALLOW_CODE_EVAL"] = "1"

# Load HumanEval dataset
code_eval_metric = evaluate.load("code_eval")


class WritableObject(object):
    "dummy output stream for pylint"
    def __init__(self):
        self.content = []
    def write(self, st):
        "dummy write"
        self.content.append(st)
    def read(self):
        "dummy read"
        return self.content

def ast_to_tree(node):
    """Convert an AST node to a zss-compatible tree."""
    if isinstance(node, ast.AST):
        children = [ast_to_tree(child) for child in ast.iter_child_nodes(node)]
        return zss.Node(type(node).__name__, children)
    else:
        return zss.Node(str(node))

def sigmoid(x, k=1, x0=0):
    """Sigmoid function to transform distance into similarity."""
    return 1 / (1 + math.exp(-k * (x - x0)))

def normalized_ast_distance(code1, code2):
    """Calculate normalized tree edit distance."""
    ast1 = ast.parse(code1)
    ast2 = ast.parse(code2)
    tree1 = ast_to_tree(ast1)
    tree2 = ast_to_tree(ast2)
    distance = zss.simple_distance(tree1, tree2)
    max_distance = len(list(ast.walk(ast1))) + len(list(ast.walk(ast2)))
    return distance / max_distance

def ast_similarity(code1, code2, k=10, x0=0.5):
    """Calculate similarity between two ASTs using sigmoid."""
    distance = normalized_ast_distance(code1, code2)
    similarity = sigmoid(distance, k=k, x0=x0)
    return similarity

def evaluate_code(code, canonical_solution_ast=None):
    """Versione migliorata della funzione di valutazione."""
    # Preprocess del codice
    try:
        code = preprocess_code(code)
    except Exception as e:
        return {
            "error": f"Preprocessing failed: {str(e)}",
            "syntactic_correct": False,
            "pylint_errors": 0,
            "pylint_warnings": 0,
            "avg_complexity": 0,
            "code_length": len(code),
            "ast_similarity": 0
        }

    metrics = {
        "syntactic_correct": False,
        "pylint_errors": 0,
        "pylint_warnings": 0,
        "avg_complexity": 0,
        "code_length": len(code),
        "ast_similarity": 0
    }

    # 1. Verifica sintattica
    if not is_valid_python_code(code):
        metrics["syntactic_correct"] = False
        return metrics

    metrics["syntactic_correct"] = True

    # 2. Analisi Pylint
    with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as temp_file:
        temp_filename = temp_file.name
        temp_file.write(code)

    try:
        pylint_output = WritableObject()
        json_reporter = JSONReporter(pylint_output)
        lint.Run([
            "--disable=all",  # Disabilita tutti i check
            "--enable=error,warning",  # Abilita solo errori e warning
            "--rcfile=/dev/null",
            temp_filename
        ], reporter=json_reporter, exit=False)

        if pylint_output.content:
            pylint_results = json.loads(pylint_output.content[0])
            metrics["pylint_errors"] = len([m for m in pylint_results if m.get("type") == "error"])
            metrics["pylint_warnings"] = len([m for m in pylint_results if m.get("type") == "warning"])
    except Exception as e:
        metrics["pylint_error"] = str(e)
    finally:
        os.remove(temp_filename)

    # 3. Analisi complessità
    try:
        process = subprocess.run(['radon', 'cc', '-j', '-'],
                               input=code, text=True, capture_output=True)
        if process.returncode == 0:
            radon_output = json.loads(process.stdout)
            if isinstance(radon_output, list):
                metrics["avg_complexity"] = sum(
                    item['complexity'] for item in radon_output) / len(radon_output)
    except Exception as e:
        metrics["radon_error"] = str(e)

    # 4. Similarità AST (solo se codice canonico fornito e codice valido)
    if canonical_solution_ast and metrics["syntactic_correct"]:
        try:
            metrics["ast_similarity"] = 1.0 - float(ast_similarity(code, canonical_solution_ast))
        except Exception as e:
            metrics["ast_similarity_error"] = str(e)

    return metrics

def extract_python_code(text: str) -> str:
    """
    Extracts Python code enclosed in triple backticks (```python ... ```) from a string.

    Args:
        text: The input string.

    Returns:
        The extracted Python code as a string, or the original text if no such block is found.
    """
    match = re.search(r"```python\n(.*?)\n```", text, re.DOTALL)  # Use DOTALL to match across lines
    if match:
        return match.group(1).strip()  # .strip() to remove leading/trailing whitespace
    return text

# Example usage

#code1 = """
#def foo(x):
#    return x + 1
#"""

#code2 = """
#def bar(y):
 #   return y - 1
#"""

#a= evaluate_code(code1, code2)
#print(a)


In [ ]:
# Store evaluation results
base_eval_results = []
fine_tuned_eval_results = []

for eval_item, non_fine_tuned, fine_tuned in zip(dataset, base_results, fine_tuned_results):
    prompt = non_fine_tuned['prompt']
    canonical = extract_python_code(eval_item['response'])

    base_pass = evaluate_code(non_fine_tuned["code"], canonical)
    fine_tuned_pass = evaluate_code(fine_tuned["code"], canonical)
    # Store results
    base_eval_results.append({"prompt": prompt, "code": non_fine_tuned["code"], **base_pass})
    fine_tuned_eval_results.append({"prompt": prompt, "code": fine_tuned["code"], **fine_tuned_pass})

In [ ]:
import pandas as pd
from tabulate import tabulate

# Convert the evaluation results to DataFrames
base_df = pd.DataFrame(base_eval_results)
fine_tuned_df = pd.DataFrame(fine_tuned_eval_results)

# Filter rows based on the specified conditions
base_df_filtered = base_df[
    (base_df["syntactic_correct"] == True) |
    (base_df["pylint_errors"] == 0) |
    (base_df["ast_similarity"] > 0.8)
]

fine_tuned_df_filtered = fine_tuned_df[
    (fine_tuned_df["syntactic_correct"] == True) |
    (fine_tuned_df["pylint_errors"] == 0) |
    (fine_tuned_df["ast_similarity"] > 0.8)
]

# Customize columns to display (including 'prompt' and 'code')
columns_to_display = ["prompt", "code", "syntactic_correct", "pylint_errors", "pylint_warnings", "avg_complexity", "code_length", "ast_similarity"]

# Display Base Model Results using Tabulate (filtered)
print("Base Model Results (filtered):")
if not base_df_filtered.empty:  # Check if the filtered DataFrame is empty
    print(tabulate(base_df_filtered[columns_to_display], headers='keys', tablefmt='grid', showindex=False))
else:
    print("No results to display (no rows match the criteria).")

# Display Fine-Tuned Model Results using Tabulate (filtered)
print("\nFine-Tuned Model Results (filtered):")
if not fine_tuned_df_filtered.empty:
    print(tabulate(fine_tuned_df_filtered[columns_to_display], headers='keys', tablefmt='grid', showindex=False))
else:
    print("No results to display (no rows match the criteria).")

# Display lengths of the filtered DataFrames
print(f"\nLength of Base Model Results (filtered): {len(base_df_filtered)}")
print(f"Length of Fine-Tuned Model Results (filtered): {len(fine_tuned_df_filtered)}")

Base Model Results (filtered):
+------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------+-----------------+-------------------+------------------+---------------+------------------+
| prompt                                                                                                           | code                                                                                                                                                                                                               | syntactic_correct   |   pylint_errors |   pylint_warnings |   avg_complexity |   code_length |   ast_similarity |
+================================================================================

In [ ]:
import ast
import radon.complexity
import subprocess
import json
from pylint import lint
from pylint.reporters.json_reporter import JSONReporter
import tempfile
import zss
import re
import os
import pandas as pd
from tabulate import tabulate
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from datasets import load_dataset
import torch
import random
import gc

# Configurazione
os.environ["HF_ALLOW_CODE_EVAL"] = "1"
base_model_name = "bigcode/starcoder2-3b"
device = "cuda" if torch.cuda.is_available() else "cpu"

class CodeEvaluator:
    def __init__(self):
        self.tokenizer = None
        self.model = None

    def initialize_models(self):
        """Inizializza i modelli e il tokenizer"""
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16,
        )

        self.tokenizer = AutoTokenizer.from_pretrained(base_model_name)
        self.model = AutoModelForCausalLM.from_pretrained(
            base_model_name,
            device_map={"": device},
            quantization_config=quantization_config
        ).eval()

    def preprocess_code(self, code: str) -> str:
        """Pulisce e prepara il codice per l'analisi"""
        if not code:
            return ""

        # Estrai solo la prima funzione definita
        matches = re.findall(r'(def\s+\w+\(.*?\):.*?(?=\n\s*def\s|\Z))', code, re.DOTALL)
        if matches:
            code = matches[0]

        # Rimuovi parti incomplete e commenti
        code = re.sub(r'#.*', '', code)  # Rimuovi commenti
        code = re.sub(r'\.\.\..*', '', code)  # Rimuovi ... e tutto ciò che segue
        code = re.sub(r'\s+\n', '\n', code)  # Rimuovi righe vuote
        code = code.strip()

        return code if code else ""

    def is_valid_python(self, code: str) -> bool:
        """Verifica se il codice è sintatticamente valido"""
        try:
            ast.parse(code)
            return True
        except (SyntaxError, IndentationError, ValueError):
            return False

    def run_pylint_analysis(self, code: str) -> dict:
        """Esegue l'analisi Pylint sul codice"""
        pylint_results = {"errors": 0, "warnings": 0}

        with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as temp_file:
            temp_filename = temp_file.name
            temp_file.write(code)

        try:
            pylint_output = []
            reporter = JSONReporter(pylint_output.append)
            lint.Run([
                "--disable=all",
                "--enable=error,warning",
                "--rcfile=/dev/null",
                temp_filename
            ], reporter=reporter, exit=False)

            if pylint_output:
                try:
                    results = json.loads(pylint_output[0])
                    pylint_results["errors"] = len([m for m in results if m.get("type") == "error"])
                    pylint_results["warnings"] = len([m for m in results if m.get("type") == "warning"])
                except json.JSONDecodeError:
                    pass
        except Exception:
            pass
        finally:
            os.remove(temp_filename)

        return pylint_results

    def calculate_complexity(self, code: str) -> float:
        """Calcola la complessità ciclomatica"""
        try:
            process = subprocess.run(
                ['radon', 'cc', '-j', '-'],
                input=code,
                text=True,
                capture_output=True,
                timeout=5
            )

            if process.returncode == 0:
                radon_output = json.loads(process.stdout)
                if isinstance(radon_output, list):
                    complexities = [item['complexity'] for item in radon_output]
                    return sum(complexities) / len(complexities) if complexities else 0
        except Exception:
            pass
        return 0

    def calculate_ast_similarity(self, code1: str, code2: str) -> float:
        """Calcola la similarità tra due AST"""
        try:
            ast1 = ast.parse(code1)
            ast2 = ast.parse(code2)

            tree1 = zss.Node("root", [self.ast_to_node(n) for n in ast.walk(ast1)])
            tree2 = zss.Node("root", [self.ast_to_node(n) for n in ast.walk(ast2)])

            distance = zss.simple_distance(tree1, tree2)
            max_size = max(len(list(ast.walk(ast1))), len(list(ast.walk(ast2))))

            return 1 - (distance / max_size) if max_size > 0 else 0
        except Exception:
            return 0

    def ast_to_node(self, node):
        """Converti un nodo AST in un nodo ZSS"""
        if isinstance(node, ast.AST):
            children = [self.ast_to_node(child) for child in ast.iter_child_nodes(node)]
            return zss.Node(type(node).__name__, children)
        return zss.Node(str(node))

    def evaluate_code(self, code: str, canonical: str = None) -> dict:
        """Valuta il codice generato"""
        metrics = {
            "syntactic_correct": False,
            "pylint_errors": 0,
            "pylint_warnings": 0,
            "avg_complexity": 0,
            "code_length": 0,
            "ast_similarity": 0,
            "has_return": False,
            "implementation_complete": False
        }

        if not code or not isinstance(code, str):
            return metrics

        # Preprocessing
        clean_code = self.preprocess_code(code)
        metrics["code_length"] = len(clean_code)

        if not clean_code:
            return metrics

        # Validità sintattica
        metrics["syntactic_correct"] = self.is_valid_python(clean_code)

        if not metrics["syntactic_correct"]:
            return metrics

        # Analisi Pylint
        pylint_results = self.run_pylint_analysis(clean_code)
        metrics["pylint_errors"] = pylint_results["errors"]
        metrics["pylint_warnings"] = pylint_results["warnings"]

        # Complessità ciclomatica
        metrics["avg_complexity"] = self.calculate_complexity(clean_code)

        # Similarità AST
        if canonical:
            metrics["ast_similarity"] = self.calculate_ast_similarity(clean_code, canonical)

        # Completezza implementazione
        metrics["has_return"] = "return" in clean_code.lower()
        metrics["implementation_complete"] = metrics["has_return"] or any(
            kw in clean_code.lower() for kw in ["yield", "raise", "pass"])

        return metrics

    def generate_code(self, prompt: str) -> str:
        """Genera codice dal prompt"""
        inputs = self.tokenizer(prompt, return_tensors="pt").to(device)

        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=200,
                pad_token_id=self.tokenizer.eos_token_id,
                do_sample=True,
                temperature=0.7
            )

        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

    def run_evaluation(self, num_examples=10):
        """Esegue la valutazione completa"""
        # Carica dataset
        humaneval = load_dataset("openai/openai_humaneval", split="test")
        sampled_humaneval = random.sample(list(humaneval), num_examples)

        # Inizializza modelli
        self.initialize_models()

        # Risultati
        results = {"base": [], "fine_tuned": []}

        for example in sampled_humaneval:
            prompt = example['prompt']
            canonical = example['canonical_solution']

            # Genera e valuta codice base
            base_code = self.generate_code(prompt)
            base_metrics = self.evaluate_code(base_code, canonical)

            results["base"].append({
                "prompt": prompt[:100] + "..." if len(prompt) > 100 else prompt,
                "code": base_code[:200] + "..." if len(base_code) > 200 else base_code,
                **base_metrics
            })

            # Placeholder per fine-tuned (sostituire con il tuo modello)
            fine_tuned_metrics = self.evaluate_code(base_code, canonical)
            results["fine_tuned"].append({
                "prompt": prompt[:100] + "..." if len(prompt) > 100 else prompt,
                "code": base_code[:200] + "..." if len(base_code) > 200 else base_code,
                **fine_tuned_metrics
            })

            gc.collect()

        # Analisi risultati
        self.analyze_results(results)

    def analyze_results(self, results):
        """Analizza e visualizza i risultati"""
        base_df = pd.DataFrame(results["base"])
        fine_tuned_df = pd.DataFrame(results["fine_tuned"])

        # Metriche aggregate
        print("\n=== Metriche Aggregate ===")
        agg_metrics = [
            ["Sintatticamente Corretto",
             base_df["syntactic_correct"].mean(),
             fine_tuned_df["syntactic_correct"].mean()],
            ["Errori Pylint (media)",
             base_df["pylint_errors"].mean(),
             fine_tuned_df["pylint_errors"].mean()],
            ["Complessità Ciclomatica",
             base_df["avg_complexity"].mean(),
             fine_tuned_df["avg_complexity"].mean()],
            ["Similarità AST",
             base_df["ast_similarity"].mean(),
             fine_tuned_df["ast_similarity"].mean()],
            ["Implementazione Completa",
             base_df["implementation_complete"].mean(),
             fine_tuned_df["implementation_complete"].mean()]
        ]

        print(tabulate(agg_metrics,
                      headers=["Metrica", "Base", "Fine-Tuned"],
                      floatfmt=".2f",
                      tablefmt="grid"))

        # Esempi dettagliati
        print("\n=== Esempi Dettagliati (solo sintatticamente corretti) ===")
        valid_examples = 0

        for i, (base, fine_tuned) in enumerate(zip(results["base"], results["fine_tuned"])):
            if not base["syntactic_correct"] and not fine_tuned["syntactic_correct"]:
                continue

            valid_examples += 1
            print(f"\nEsempio {valid_examples}:")
            print(f"Prompt: {base['prompt']}")
            print("\nBase Model:")
            print(f"Codice: {base['code']}")
            print("Metriche:")
            for k, v in base.items():
                if k not in ['prompt', 'code']:
                    print(f"- {k}: {v}")

            print("\nFine-Tuned Model:")
            print(f"Codice: {fine_tuned['code']}")
            print("Metriche:")
            for k, v in fine_tuned.items():
                if k not in ['prompt', 'code']:
                    print(f"- {k}: {v}")

        if valid_examples == 0:
            print("Nessun esempio sintatticamente corretto trovato.")

if __name__ == "__main__":
    evaluator = CodeEvaluator()
    evaluator.run_evaluation(num_examples=5)

No files to lint: exiting.


SystemExit: 32